In [209]:
import numpy as np
from scipy import optimize
import pprint
import math
import time

In [210]:
# Load data
data = np.loadtxt("CASP.csv", delimiter = ",", skiprows = 1)
y = data[ : , 0 ]
X = data[ : , 1 : ]

In [211]:
# Function to split the data into first fraction_train% and remaining as test.
def split_train_test(X, y, fraction_train = 9.0 / 10.0):
    end_train = round(X.shape[ 0 ] * fraction_train)
    X_train = X[ 0 : end_train, ]
    y_train = y[ 0 : end_train ]
    X_test = X[ end_train :, ]
    y_test = y[ end_train : ]
    return X_train, y_train, X_test, y_test

# Given a test and feature set, normalize each so mean is 0 and unit deviation.
def normalize_features(X_train, X_test):
    mean_X_train = np.mean(X_train, 0)
    std_X_train = np.std(X_train, 0)
    std_X_train[ std_X_train == 0 ] = 1
    X_train_normalized = (X_train - mean_X_train) / std_X_train
    X_test_normalized = (X_test - mean_X_train) / std_X_train
    return X_train_normalized, X_test_normalized

In [212]:
# Clean data
X_train, y_train, X_test, y_test = split_train_test(X, y)
X_train, X_test = normalize_features(X_train, X_test)
X_train = np.concatenate((X_train, np.ones((X_train.shape[ 0 ], 1))), 1)
X_test = np.concatenate((X_test, np.ones((X_test.shape[ 0 ], 1))), 1)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:4: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:5: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:6: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:7: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [213]:
# Function which finds the MAP estimate.
def mapEstimate(Xtrain, Ytrain, lam=0):
    """
    Ridge regression performed on the given design matrix (Xtrain) and results (Ytrain). 
    Essentially, we implement the algorithm described in 7.5.2 of Murphy.
    
    lam is lambda=sigma^2/tau^2 where sigma is variance of data and tau is prior variance.
    
    Returns the mapEstimate of the paramters w.
    """
    (N,D) = Xtrain.shape
    Ny = Ytrain.shape
    assert(N == Ny[0])
    
    # Attach additional prior data to Xtrain.
    priorData = np.diag([np.sqrt(lam) for _ in xrange(D)])
    Xtrainp = np.concatenate((Xtrain, priorData))
    Ytrainp = np.concatenate((Ytrain, np.zeros(D)))
    
    # Find QR decomposition of Xp to avoid inverse.
    Q,R = np.linalg.qr(Xtrainp)
    inverseR = np.linalg.inv(R)
    return inverseR.dot(Q.T).dot(Ytrainp)

In [214]:
# Calculate MAP estimate of W
variance = 1.0
priorVariance = (.1)**2
wMAP = mapEstimate(X_train, y_train, lam=variance / priorVariance)

In [215]:
pprint.pprint(wMAP)

array([ 3.15915374,  2.45711946,  1.00708615, -5.70882509,  0.19052137,
       -1.49631801, -0.25545196,  0.82089849, -0.61340907,  7.72464619])


In [216]:
# Predict using model parameters W
def predict(X,w):
    return X.dot(w)

# Calculate root mean square error
def RMS(pred, target):
    return np.sqrt(((pred - target) ** 2).mean())

In [217]:
# Actually print the RMSE
print RMS(predict(X_test,wMAP), y_test)

5.21279653606


In [218]:
# Function to calculate the gradiant of the posterior on W.
def gradiantPosterior(w, X, Y, lam2, tau2):
    return 1.0/lam2 * X.T.dot(Y - X.dot(w)) - 1.0/tau2 * w

def gradiant(w):
    return gradiantPosterior(w, X_train, y_train, variance, priorVariance)

In [219]:
# Evaluates logPosterior up to an additive constant.
def logPosterior(w, X, Y, lam2, tau2):
    (N,D) = X.shape
    var0 = -(1. /(2 * lam2)) * (Y - X.dot(w)).T.dot(Y - X.dot(w))
    var1 = -(1. / (2 * tau2)) * w.T.dot(w)
    return var0 + var1
def logDist(w):
    return logPosterior(w, X_train, y_train, variance, priorVariance)

In [220]:
def obj_and_gradiant(w):
    return (logDist(w), gradiant(w))

In [221]:
def verifyGradiant(x, epsilon = 0.0001):
    """
    Uses finite differences to verify the gradiant of our log posterior.
    """
    units = np.diag([epsilon for _ in x])
    grad = gradiant(x)
    finiteDiff = [(logDist(x + units[i]) - logDist(x - units[i])) / (2*epsilon) for i in xrange(len(x))]
    return all([abs(g - f) < 0.01 for g,f in zip(grad, finiteDiff)])

In [222]:
# Verify the result for ntrials
def verifyGradiantM(ntrials):
    xs = [np.random.rand(D) for _ in xrange(ntrials)]
    return all(map(verifyGradiant, xs))

In [223]:
print verifyGradiantM(100)

True


In [224]:
def minimizePosterior(D):
    return optimize.minimize(lambda x: (-1*logDist(x),gradiant(x)), np.zeros(D),method="L-BFGS-B",
                             jac=True, options={'maxiter': 100, 'disp' : True})

In [225]:
def minimizePosteriorGeneral(D,X,Y,lam2,tau2):
    return optimize.minimize(lambda x: (-1*logPosterior(x,X,Y,lam2,tau2), gradiantPosterior(x,X,Y,lam2,tau2)),
                             np.zeros(D),method="L-BFGS-B", jac=True, options={'maxiter': 100, 'disp' : True})

In [226]:
wMin = minimizePosterior(wMAP.shape[0]).x
print wMin, wMAP

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.] [ 3.15915374  2.45711946  1.00708615 -5.70882509  0.19052137 -1.49631801
 -0.25545196  0.82089849 -0.61340907  7.72464619]


In [227]:
# Actually print the RMSE of this new estimate
print RMS(predict(X_test,wMin), y_test)

9.92540584064


In [228]:
y_train.std()

6.1152647488436624

In [229]:
# Given a target dimension d and an input dimension D of the features,
# constructs random non-linear functions to take each data point from d -> D dimensions.
def genNonLinearFunctions(d,D):
    A = np.random.multivariate_normal(np.zeros(D),np.identity(D), d)
    b = np.random.uniform(high=2*math.pi, size=(d,1))
    return lambda x : np.cos(A.dot(x) + b)

In [230]:
dvals = [100, 200, 400, 600]
(N,D) = X_train.shape
funs = [genNonLinearFunctions(d, D) for d in dvals]

In [ ]:
newDesignMatrices = []
for f in funs:
    # The new N x d design matrices.
    newX_train, newX_test = f(X_train.T).T, f(X_test.T).T
    
    # We renormalize in the new space
    newX_train, newX_test = normalize_features(newX_train, newX_test)
    
    # We append a bias term.
    newX_train = np.concatenate((newX_train, np.ones((newX_train.shape[ 0 ], 1))), 1)
    newX_test = np.concatenate((newX_test, np.ones((newX_test.shape[ 0 ], 1))), 1)
    
    # Append to our list so we don't recalculate in the future
    newDesignMatrices.append((newX_train, newX_test)) 

In [ ]:
# We now record timing and results in a dictionary.
results = []
for (i,d) in enumerate(dvals):
    result = {'d' : d }
    
    # QR Decomp
    start = time.time()
    train, test = newDesignMatrices[i]
    wMAP = mapEstimate(train, y_train, lam=variance / priorVariance)
    result['qr_time'] = time.time() - start
    error = RMS(predict(test, wMAP), y_test)
    result['qr_error'] = error
    
    # L-BFGS
    start = time.time()
    wMin = minimizePosteriorGeneral(wMAP.shape[0], train, y_train, variance, priorVariance).x
    lbfgs = time.time() - start
    result['lbgfs_time'] = time.time() - start
    error = RMS(predict(test, wMin), y_test)
    result['lbgfs_error'] = error
    results.append(result)

In [ ]:
# We now plot the results.
qrx, qry, lbgfsx, lbgfsy = [], [], [], []
for result in results:
    qrx.append(result['qr_time']) 
    qry.append(result['qr_error'])
    lbgfsx.append(result['lbgfs_time'])
    lbgfsy.append(result['lbgfs_error'])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
qrplot = plt.scatter(qrx, qry, color="red")
# Anotetate the points with their respective d-values
for i in range(len(dvals)):
    plt.annotate("d={}".format(dvals[i]), (qrx[i],qry[i]), textcoords = 'offset points', ha = 'right', 
                 va = 'bottom')
lbgsfsplot = plt.scatter(lbgfsx, lbgfsy, color="blue")
  
plt.xlabel("Execution Time (s)")
plt.ylabel("Root Mean Square Error")
plt.title("RMSE vs Time for Least Square Ridge Regression")
plt.legend((qrplot, lbgsfsplot),
           ('QR Decomposition Method', 'Maximum A Posteriori'),
           scatterpoints=1,
           loc='top right',
           ncol=2,
           fontsize=10)
    
plt.show()